<a href="https://colab.research.google.com/github/ogadepoju/MachineLearning/blob/master/Ebayipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 Step 1: Setup Environment
!pip install transformers datasets seqeval accelerate --quiet


import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric
from sklearn.model_selection import train_test_split
from seqeval.metrics import classification_report, f1_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.0 MB/s eta 0:00:00


In [ ]:
# 📌 Step 2: Load & Prepare Data
# Replace with your actual file paths
train_file_path = "/content/Tagged_Titles_Train.tsv"  # train data with: Record Number, Category Id, Title, Token, Tag

df = pd.read_csv(train_file_path, sep='\t', keep_default_na=False, na_values=None)
df['Tag'].replace('', np.nan, inplace=True)
df['Tag'].fillna(method='ffill', inplace=True)  # Fill continuation tags (multi-token entities)

# Group tokens by sentence (title)
grouped = df.groupby("Record Number").agg({
    "Title": "first",
    "Token": list,
    "Tag": list
}).reset_index()


/tmp/ipython-input-2-3458559437.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Tag'].replace('', np.nan, inplace=True)
/tmp/ipython-input-2-3458559437.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [ ]:
# 📌 Step 3: Encode Tags
unique_tags = sorted(set(tag for tags in grouped["Tag"] for tag in tags))
tag2id = {tag: idx for idx, tag in enumerate(unique_tags)}
id2tag = {idx: tag for tag, idx in tag2id.items()}
print("Tag map:", tag2id)

# Load tokenizer
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding="max_length") # Added padding
    labels = []
    for i, label in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(tag2id[label[word_idx]])  # Use same tag (can adjust for BIO if needed)
            previous_word_idx = word_idx
        # Pad label_ids to the same length as tokenized_inputs
        while len(label_ids) < len(tokenized_inputs["input_ids"][i]):
            label_ids.append(-100)
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Build HuggingFace Dataset
dataset = Dataset.from_pandas(grouped.rename(columns={"Token": "tokens", "Tag": "labels"}))
dataset = dataset.train_test_split(test_size=0.1)
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Tag map: {'Anwendung': 0, 'Anzahl_Der_Einheiten': 1, 'Besonderheiten': 2, 'Breite': 3, 'Bremsscheiben-Aussendurchmesser': 4, 'Bremsscheibenart': 5, 'Einbauposition': 6, 'Farbe': 7, 'Größe': 8, 'Hersteller': 9, 'Herstellernummer': 10, 'Herstellungsland_Und_-Region': 11, 'Im_Lieferumfang_Enthalten': 12, 'Kompatible_Fahrzeug_Marke': 13, 'Kompatibles_Fahrzeug_Jahr': 14, 'Kompatibles_Fahrzeug_Modell': 15, 'Länge': 16, 'Material': 17, 'Maßeinheit': 18, 'Menge': 19, 'Modell': 20, 'O': 21, 'Oberflächenbeschaffenheit': 22, 'Oe/Oem_Referenznummer(N)': 23, 'Produktart': 24, 'Produktlinie': 25, 'SAE_Viskosität': 26, 'Stärke': 27, 'Technologie': 28, 'Zähnezahl': 29}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# 📌 Step 4: Load Model and Train
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(tag2id))

args = TrainingArguments(
    output_dir="./ner-ebay",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4-2817386359.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ogadepoju (ogadepoju-university-of-cincinnati) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.498300,0.196782
2,0.191700,0.178922
3,0.159200,0.170752
4,0.132800,0.174380
5,0.112600,0.177575


TrainOutput(global_step=2815, training_loss=0.20687089313622267, metrics={'train_runtime': 2722.6848, 'train_samples_per_second': 8.264, 'train_steps_per_second': 1.034, 'total_flos': 5880665318400000.0, 'train_loss': 0.20687089313622267, 'epoch': 5.0})

In [ ]:
# 📌 Step 5: Evaluate
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
preds = np.argmax(predictions, axis=2)

true_labels = [[id2tag[label] for label in sent if label != -100] for sent in labels]
true_preds = [[id2tag[p] for (p, l) in zip(pred, sent_labels) if l != -100] for pred, sent_labels in zip(preds, labels)]

print(classification_report(true_labels, true_preds))


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Produktart seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Kompatible_Fahrzeug_Marke seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Kompatibles_Fahrzeug_Modell seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Hersteller seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Herstellernummer seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/l

                            precision    recall  f1-score   support

         Aussendurchmesser       0.96      0.99      0.98       324
                      arbe       0.00      0.00      0.00         1
                   aterial       1.00      1.00      1.00         5
                 aßeinheit       1.00      0.98      0.99        46
                      enge       0.00      0.00      0.00         0
                 ersteller       0.99      0.98      0.99       422
           erstellernummer       0.87      0.90      0.88       140
             esonderheiten       0.43      0.23      0.30        13
             inbauposition       0.99      1.00      1.00       720
  m_Lieferumfang_Enthalten       0.93      0.92      0.92       559
                  nwendung       1.00      0.50      0.67         2
       nzahl_Der_Einheiten       0.94      1.00      0.97        31
                     odell       0.44      0.75      0.56        16
  ompatible_Fahrzeug_Marke       0.95      0.97

In [ ]:
# 🧠 Clean and Safe NER Pipeline for eBay Competition

import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from datasets import Dataset

# Define aspect names from competition (O is excluded in submission)
VALID_ASPECTS = [
    'Anwendung', 'Anzahl_Der_Einheiten', 'Besonderheiten', 'Breite',
    'Bremsscheiben-Aussendurchmesser', 'Bremsscheibenart', 'Einbauposition',
    'Farbe', 'Größe', 'Hersteller', 'Herstellernummer',
    'Herstellungsland_Und_-Region', 'Im_Lieferumfang_Enthalten',
    'Kompatible_Fahrzeug_Marke', 'Kompatibles_Fahrzeug_Jahr', 'Kompatibles_Fahrzeug_Modell',
    'Länge', 'Material', 'Maßeinheit', 'Menge', 'Modell', 'Oberflächenbeschaffenheit',
    'Oe/Oem_Referenznummer(N)', 'Produktart', 'Produktlinie', 'SAE_Viskosität',
    'Stärke', 'Technologie', 'Zähnezahl'
]

# ⚙️ Clean and fix broken label predictions

def clean_prediction_label(label):
    for aspect in VALID_ASPECTS:
        if label.lower() in aspect.lower():
            return aspect
    return None  # Invalid prediction

# 🧱 Token-to-entity conversion (BIO -> phrases)

def extract_predicted_aspects(record_id, category_id, tokens, tags):
    aspects = []
    current_aspect = None
    current_tokens = []
    current_label = None

    for token, tag in zip(tokens, tags):
        if tag == 'O':
            if current_aspect:
                aspect_value = " ".join(current_tokens)
                fixed_label = clean_prediction_label(current_label)
                if fixed_label:
                    aspects.append({
                        'record_number': record_id,
                        'category_id': category_id,
                        'aspect_name': fixed_label,
                        'aspect_value': aspect_value
                    })
                current_aspect = None
                current_tokens = []
                current_label = None
            continue

        prefix, label = tag.split('-', 1) if '-' in tag else ('B', tag)

        if prefix == 'B' or current_label != label:
            if current_tokens and current_label:
                aspect_value = " ".join(current_tokens)
                fixed_label = clean_prediction_label(current_label)
                if fixed_label:
                    aspects.append({
                        'record_number': record_id,
                        'category_id': category_id,
                        'aspect_name': fixed_label,
                        'aspect_value': aspect_value
                    })
            current_tokens = [token]
            current_label = label
        else:
            current_tokens.append(token)

    if current_tokens and current_label:
        aspect_value = " ".join(current_tokens)
        fixed_label = clean_prediction_label(current_label)
        if fixed_label:
            aspects.append({
                'record_number': record_id,
                'category_id': category_id,
                'aspect_name': fixed_label,
                'aspect_value': aspect_value
            })

    return aspects

# 🧾 Final TSV Submission Formatter

def generate_submission(predictions, metadata_df, output_path="submission.tsv"):
    sub_df = pd.DataFrame(predictions)
    required_columns = ['record_number', 'category_id', 'aspect_name', 'aspect_value']
    sub_df = sub_df[required_columns]
    sub_df.columns = ['RecordNumber', 'CategoryId', 'AspectName', 'AspectValue']

    # Validate
    quiz_ids = set(metadata_df['Record Number'].values)
    predicted_ids = set(sub_df['RecordNumber'].unique())
    missing = quiz_ids - predicted_ids
    print(f"⚠️ Missing {len(missing)} quiz records in submission.")

    # Save
    sub_df.to_csv(output_path, sep='\t', index=False, encoding='utf-8', quoting=3)
    print(f"✅ Submission saved to: {output_path}")
